In [1]:
using LinearAlgebra
using SparseArrays
using BenchmarkTools
using Random

In [ ]:
# Projection error
train_proj_err = zeros(rmax - (rmin-1), burger.Pdim) 
# Relative state error
train_state_err = Dict(
    :int => zeros(rmax-rmin+1, burger.Pdim),
    :LS => zeros(rmax-rmin+1, burger.Pdim),
    :ephec => zeros(rmax-rmin+1, burger.Pdim),
    :epsic => zeros(rmax-rmin+1, burger.Pdim),
    :epuc => zeros(rmax-rmin+1, burger.Pdim),
)
# Energy 
train_En = Dict(
    :int => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :LS => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :ephec => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :epsic => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :epuc => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :fom => Array{VecOrMat{Float64}}(undef, burger.Pdim)
)
# Constraint Residual
train_CR = Dict(
    :int => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :LS => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :ephec => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :epsic => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :epuc => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :fom => Array{Float64}(undef, burger.Pdim)
)
# Momentum
train_mmt = Dict(
    :int => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :LS => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :ephec => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :epsic => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :epuc => Array{Float64}(undef, rmax - (rmin - 1), burger.Pdim),
    :fom => Array{Float64}(undef, burger.Pdim)
)
# Constraint Violation
train_CV = Dict(
    :int => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :LS => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :ephec => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :epsic => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :epuc => Array{VecOrMat{Float64}}(undef, rmax - (rmin - 1), burger.Pdim),
    :fom => Array{VecOrMat{Float64}}(undef, burger.Pdim)
)

# Load the data dictionary
if !(@isdefined Data)
    @info "Loading data from $(filename)"
    if !isfile(filename)
        error("Run the cell above to generate the data first!")
    end
    Data = load(filename)
    op_LS = Data["op_LS"]
    op_int = Data["op_int"]
    op_ephec = Data["op_ephec"]
    op_epsic = Data["op_epsic"]
    op_epuc = Data["op_epuc"]
    
    Xtr_all = Data["Xtr_all"]
    Vrmax = Data["Vrmax"]
    @info "Data loaded."
end

@info "Analyze the operators with training data..."
@showprogress for i in 1:length(burger.μs)

    # Energy, constraint residual, and constraint violation of the FOM
    train_En[:fom][i] = norm.(eachcol(Xtr_all[i]), 2)
    F_full = Data["op_fom_tr"][i].F
    train_CR[:fom][i], train_mmt[:fom][i] = LnL.constraintResidual(F_full, size(F_full, 1), options.optim.which_quad_term)
    train_CV[:fom][i] = LnL.constraintViolation(Xtr_all[i], F_full, options.optim.which_quad_term)
    
    for (j,r) in enumerate(rmin:rmax)
        Vr = Vrmax[i][:, 1:r]

        # Temporary data storage
        PE = zeros(num_ic_params)  # projection error
        SE = Dict(
            :int => zeros(num_ic_params),
            :LS => zeros(num_ic_params),
            :ephec => zeros(num_ic_params),
            :epsic => zeros(num_ic_params),
            :epuc => zeros(num_ic_params),
        )
        En = Dict(
            :int => Array{Float64}(undef, num_ic_params, burger.Tdim),
            :LS => Array{Float64}(undef, num_ic_params, burger.Tdim),
            :ephec => Array{Float64}(undef, num_ic_params, burger.Tdim),
            :epsic => Array{Float64}(undef, num_ic_params, burger.Tdim),
            :epuc => Array{Float64}(undef, num_ic_params, burger.Tdim),
        )
        CR = Dict(
            :int => Array{Float64}(undef, num_ic_params),
            :LS => Array{Float64}(undef, num_ic_params),
            :ephec => Array{Float64}(undef, num_ic_params),
            :epsic => Array{Float64}(undef, num_ic_params),
            :epuc => Array{Float64}(undef, num_ic_params),
        )
        mmt = Dict(
            :int => Array{Float64}(undef, num_ic_params),
            :LS => Array{Float64}(undef, num_ic_params),
            :ephec => Array{Float64}(undef, num_ic_params),
            :epsic => Array{Float64}(undef, num_ic_params),
            :epuc => Array{Float64}(undef, num_ic_params),
        )
        CV = Dict(
            :int => Array{Float64}(undef, num_ic_params, burger.Tdim),
            :LS => Array{Float64}(undef, num_ic_params, burger.Tdim),
            :ephec => Array{Float64}(undef, num_ic_params, burger.Tdim),
            :epsic => Array{Float64}(undef, num_ic_params, burger.Tdim),
            :epuc => Array{Float64}(undef, num_ic_params, burger.Tdim),
        )

        ct = 1  # set/reset counter
        for amp in AMP, freq in FREQ, phase in PHASE
            IC = init_wave(amp, freq, phase)

            # Integrate the LS operator inference model
            Finf_extract_LS = LnL.extractF(op_LS[i].F, r)
            Xinf_LS = burger.semiImplicitEuler(op_LS[i].A[1:r, 1:r], Finf_extract_LS, burger.t, Vr' * IC)

            # Integrate the intrusive model
            Fint_extract = LnL.extractF(op_int[i].F, r)
            Xint = burger.semiImplicitEuler(op_int[i].A[1:r, 1:r], Fint_extract, burger.t, Vr' * IC)
            
            if options.optim.SIGE
                # Integrate the energy-preserving hard equality constraint operator inference model
                Finf_extract_ephec = op_ephec[i,j].F
                Xinf_ephec = burger.semiImplicitEuler(op_ephec[i,j].A, op_ephec[i,j].F, burger.t, Vr' * IC)
                # Finf_extract_ephec = LnL.extractF(op_ephec[i,rmax].F, r)
                # Xinf_ephec = burger.semiImplicitEuler(op_ephec[i,rmax].A[1:r, 1:r], Finf_extract_ephec, burger.t, Vr' * IC)
                
                # Integrate the energy-preserving soft inequality constraint operator inference model
                Finf_extract_epsic = op_epsic[i,j].F
                Xinf_epsic = burger.semiImplicitEuler(op_epsic[i,j].A, op_epsic[i,j].F, burger.t, Vr' * IC)
                # Finf_extract_epsic = LnL.extractF(op_epsic[i,rmax].F, r)
                # Xinf_epsic = burger.semiImplicitEuler(op_epsic[i,rmax].A[1:r, 1:r], Finf_extract_epsic, burger.t, Vr' * IC)

                # Integrate the energy-preserving unconstrained operator inference model
                Finf_extract_epuc = op_epuc[i,j].F
                Xinf_epuc = burger.semiImplicitEuler(op_epuc[i,j].A, op_epuc[i,j].F, burger.t, Vr' * IC)
                # Finf_extract_epuc = LnL.extractF(op_epuc[i,rmax].F, r)
                # Xinf_epuc = burger.semiImplicitEuler(op_epuc[i,rmax].A[1:r, 1:r], Finf_extract_epuc, burger.t, Vr' * IC)
            else
                # Integrate the energy-preserving hard equality constraint operator inference model
                Finf_extract_ephec = LnL.extractF(op_ephec[i].F, r)
                Xinf_ephec = burger.semiImplicitEuler(op_ephec[i].A[1:r, 1:r], Finf_extract_ephec, burger.t, Vr' * IC)
                
                # Integrate the energy-preserving soft inequality constraint operator inference model
                Finf_extract_epsic = LnL.extractF(op_epsic[i].F, r)
                Xinf_epsic = burger.semiImplicitEuler(op_epsic[i].A[1:r, 1:r], Finf_extract_epsic, burger.t, Vr' * IC)

                # Integrate the energy-preserving unconstrained operator inference model
                Finf_extract_epuc = LnL.extractF(op_epuc[i].F, r)
                Xinf_epuc = burger.semiImplicitEuler(op_epuc[i].A[1:r, 1:r], Finf_extract_epuc, burger.t, Vr' * IC)
            end

            # Compute the projection error
            PE[ct] = LnL.compProjError(Xtr_all[i,ct], Vr)

            # Compute the relative state error
            SE[:LS][ct] = LnL.compStateError(Xtr_all[i,ct][:, 1:DS:end], Xinf_LS[:, 1:DS:end], Vr)
            SE[:int][ct] = LnL.compStateError(Xtr_all[i,ct][:, 1:DS:end], Xint[:, 1:DS:end], Vr)
            SE[:ephec][ct] = LnL.compStateError(Xtr_all[i,ct][:, 1:DS:end], Xinf_ephec[:, 1:DS:end], Vr)
            SE[:epsic][ct] = LnL.compStateError(Xtr_all[i,ct][:, 1:DS:end], Xinf_epsic[:, 1:DS:end], Vr)
            SE[:epuc][ct] = LnL.compStateError(Xtr_all[i,ct][:, 1:DS:end], Xinf_epuc[:, 1:DS:end], Vr)

            # Compute the energy
            En[:LS][ct, :] .= norm.(eachcol(Vr * Xinf_LS), 2)
            En[:int][ct, :] .= norm.(eachcol(Vr * Xint), 2)
            En[:ephec][ct, :] .= norm.(eachcol(Vr * Xinf_ephec), 2)
            En[:epsic][ct, :] .= norm.(eachcol(Vr * Xinf_epsic), 2)
            En[:epuc][ct, :] .= norm.(eachcol(Vr * Xinf_epuc), 2)

            # Compute the constraint residual and momentum
            CR[:LS][ct], mmt[:LS][ct] =  LnL.constraintResidual(Finf_extract_LS, r, options.optim.which_quad_term)
            CR[:int][ct], mmt[:int][ct] = LnL.constraintResidual(Fint_extract, r, options.optim.which_quad_term)
            CR[:ephec][ct], mmt[:ephec][ct] = LnL.constraintResidual(Finf_extract_ephec, r, options.optim.which_quad_term)
            CR[:epsic][ct], mmt[:epsic][ct] = LnL.constraintResidual(Finf_extract_epsic, r, options.optim.which_quad_term)
            CR[:epuc][ct], mmt[:epuc][ct] = LnL.constraintResidual(Finf_extract_epuc, r, options.optim.which_quad_term)

            # Compute the constraint violation
            CV[:LS][ct, :] .= LnL.constraintViolation(Xinf_LS, Finf_extract_LS, options.optim.which_quad_term)
            CV[:int][ct, :] .= LnL.constraintViolation(Xint, Fint_extract, options.optim.which_quad_term)
            CV[:ephec][ct, :] .= LnL.constraintViolation(Xinf_ephec, Finf_extract_ephec, options.optim.which_quad_term)
            CV[:epsic][ct, :] .= LnL.constraintViolation(Xinf_epsic, Finf_extract_epsic, options.optim.which_quad_term)
            CV[:epuc][ct, :] .= LnL.constraintViolation(Xinf_epuc, Finf_extract_epuc, options.optim.which_quad_term)

            # Increment counter
            ct += 1
        end

        # Compute errors
        train_proj_err[j, i] = mean(PE)
        train_state_err[:LS][j, i] = mean(SE[:LS])
        train_state_err[:int][j, i] = mean(SE[:int])
        train_state_err[:ephec][j, i] = mean(SE[:ephec])
        train_state_err[:epsic][j, i] = mean(SE[:epsic])
        train_state_err[:epuc][j, i] = mean(SE[:epuc])
        
        # Compute energy 
        train_En[:LS][j, i] = mean(En[:LS], dims=1)
        train_En[:int][j, i] = mean(En[:int], dims=1)
        train_En[:ephec][j, i] = mean(En[:ephec], dims=1)
        train_En[:epsic][j, i] = mean(En[:epsic], dims=1)
        train_En[:epuc][j, i] = mean(En[:epuc], dims=1)
        
        # Compute the CR and momentum
        train_CR[:LS][j, i] = mean(CR[:LS])
        train_CR[:int][j, i] = mean(CR[:int])
        train_CR[:ephec][j, i] = mean(CR[:ephec])
        train_CR[:epsic][j, i] = mean(CR[:epsic])
        train_CR[:epuc][j, i] = mean(CR[:epuc])

        train_mmt[:LS][j, i] = mean(mmt[:LS])
        train_mmt[:int][j, i] = mean(mmt[:int])
        train_mmt[:ephec][j, i] = mean(mmt[:ephec])
        train_mmt[:epsic][j, i] = mean(mmt[:epsic])
        train_mmt[:epuc][j, i] = mean(mmt[:epuc])

        # Compute constraint violations
        train_CV[:LS][j, i] = mean(CV[:LS], dims=1)
        train_CV[:int][j, i] = mean(CV[:int], dims=1)
        train_CV[:ephec][j, i] = mean(CV[:ephec], dims=1)
        train_CV[:epsic][j, i] = mean(CV[:epsic], dims=1)
        train_CV[:epuc][j, i] = mean(CV[:epuc], dims=1)
    end
end

# Save data
Data["train_proj_err"] = train_proj_err
Data["train_state_err"] = train_state_err
Data["train_En"] = train_En
Data["train_CR"] = train_CR
Data["train_mmt"] = train_mmt
Data["train_CV"] = train_CV
save(filename, Data)

In [2]:
A = Array{Float64}(undef, 3, 4, 5)

3×4×5 Array{Float64, 3}:
[:, :, 1] =
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312

[:, :, 2] =
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312

[:, :, 3] =
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312

[:, :, 4] =
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312
 9.64019e-312  9.64019e-312  9.64019e-312  9.64019e-312

[:, :, 5] =
 9.64019e-312  9.64019e-312  9.64019e-312  0.0
 9.64019e-312  9.64019e-312  9.64019e-312  0.0
 9.64019e-312  9.64019e-312  0.0           0.0

In [11]:
V = Vector{Vector{Float64}}(undef, 3)
for i in 1:3
    V[i] = rand(3)
end

In [12]:
for (i, v) in enumerate(V)
    println(i)
    println(v)
end

1
[0.377605558178562, 0.07934021678374625, 0.44499476993820564]
2
[0.29923360746252226, 0.33290373171816634, 0.9013376284026704]
3
[0.857420757230221, 0.05447472960431976, 0.5917774617947198]


In [8]:
A[:,[1,3,5]] = round.(rand(5,3), digits=1)

5×3 Matrix{Float64}:
 0.0  0.1  0.4
 0.8  0.7  0.2
 0.9  0.3  0.1
 0.4  0.4  0.4
 0.4  1.0  0.8

In [6]:
foo = rand(5,10)
bar = Dict(
    :pac => foo,
    :pux => ones(10,1)
)
mean(bar[:pac], dims=1)

1×10 Matrix{Float64}:
 0.829963  0.540114  0.313887  0.521348  …  0.497592  0.718996  0.481919

In [9]:
A

5×5 Matrix{Float64}:
 0.0  0.619186  0.1  0.278075   0.4
 0.8  0.255611  0.7  0.305068   0.2
 0.9  0.343694  0.3  0.0618997  0.1
 0.4  0.476888  0.4  0.590959   0.4
 0.4  0.519326  1.0  0.292222   0.8

In [11]:
[A zeros(5,1); zeros(1,6)]

6×6 Matrix{Float64}:
 0.0  0.619186  0.1  0.278075   0.4  0.0
 0.8  0.255611  0.7  0.305068   0.2  0.0
 0.9  0.343694  0.3  0.0618997  0.1  0.0
 0.4  0.476888  0.4  0.590959   0.4  0.0
 0.4  0.519326  1.0  0.292222   0.8  0.0
 0.0  0.0       0.0  0.0        0.0  0.0

In [22]:
using JuMP
using Ipopt
using Pipe

i = 2
model = Model(Ipopt.Optimizer; add_bridges=false)
@variable(model, x[1:5])

@pipe x |> [JuMP.delete(model, x), JuMP.unregister(model, :x)]
@show JuMP.is_valid(model, x)

MethodError: MethodError: objects of type Vector{Nothing} are not callable
Use square brackets [] for indexing an Array.

In [2]:
struct elemat{T} <: AbstractVector{T}
    n::Int
    i::Int

    function elemat(n::Int, i::Int)
        @boundscheck n >= i || throw(ArgumentError("$i is larger than the size $n"))
        e = spzeros(n,1)
        e[i] = 1
        return e
    end
end

"""
    elemat(n,i,j)
"""

struct Elemat{T} <: AbstractMatrix{T}
    m :: Int
    n :: Int
    i :: Int
    j :: Int
    
    function Elemat(m::Integer, n::Integer, i::Integer, j::Integer)
        return elemat(m,i) * elemat(n,j)'
    end
end

Elemat(m::Integer, i::Integer, j::Integer) = Elemat(m, m, i, j)

Elemat

In [3]:
# Kronecker delta
function delta(i::Int, j::Int)
    return Int(i == j)
end

"""
    commat(n,i,j)
"""

struct commat{T} <: AbstractMatrix{T}
    m::Int
    n::Int

    function commat(m::Integer, n::Integer)
        v = [m * (j-1) + i for i in 1:m, j in 1:n]'[:]
        # apply this permutation to the rows (i.e. to each column) of identity matrix
        K = [[delta(i, j) for j in 1:(m*n)] for i in 1:(m*n)]
        tmp = [K[i] for i in v]
        return sparse(reduce(vcat, tmp'))
    end
end

commat(m::Integer) = commat(m, m)


"""
    nommat(n,i,j)
"""

struct nommat{T} <: AbstractMatrix{T}
    m::Int
    n::Int

    function nommat(m::Integer, n::Integer)
        mn = Int(m * n)
        return 0.5 * (sparse(1.0I, mn, mn) + commat(m, n))
    end
end

nommat(m::Integer) = nommat(m, m)

nommat

In [4]:
function vech(A::AbstractMatrix{T}) where {T}
    m = LinearAlgebra.checksquare(A)
    v = Vector{T}(undef, Int(m * (m + 1) /2))
    k = 0
    for j = 1:m, i = j:m
        @inbounds v[k+=1] = A[i, j]
    end
    return v
end

vech (generic function with 1 method)

In [5]:
struct unit_DL{T} <: AbstractVector{T}
    n::Int
    i::Int
    j::Int

    function unit_DL(n::Integer, i::Integer, j::Integer)
        @boundscheck i >= j || throw(ArgumentError("Does not satisfy i >= j"))
        u = spzeros(Int(0.5 * n * (n+1)), 1)
        u[Int( (j-1)*n + i - 0.5*j*(j-1) )] = 1.0
        return u
    end
end

In [6]:
struct elimat{T} <: AbstractMatrix{T}
    m::Int
    
    function elimat(m::Integer)
        T = tril(ones(m, m)) # Lower triangle of 1's
        f = findall(x -> x == 1, T[:]) # Get linear indexes of 1's
        k = m * (m + 1) / 2 # Row size of L
        m2 = m * m # Colunm size of L
        x = f + m2 * (0:k-1) # Linear indexes of the 1's within L'

        row = [mod(a, m2) != 0 ? mod(a, m2) : m2 for a in x]
        col = [mod(a, m2) != 0 ? div(a, m2) + 1 : div(a, m2) for a in x]
        L = sparse(row, col, ones(length(x)), m2, k)
        L = L' # Now transpose to actual L
        return L
    end
end

In [11]:
N = 3
Δx = 0.5
S = Int(N * (N + 1) / 2)
Fval = repeat([1.0, 1.0, -1.0, -1.0], outer=N - 2)

4-element Vector{Float64}:
  1.0
  1.0
 -1.0
 -1.0

In [12]:
row_i = repeat(2:(N-1), inner=4)

4-element Vector{Int64}:
 2
 2
 2
 2

In [14]:
N = 4
seq = Int.([2 + (N + 1) * (x - 1) - x * (x - 1) / 2 for x in 1:(N-1)])

3-element Vector{Int64}:
 2
 6
 9

In [ ]:
col_i = vcat(seq[1], repeat(seq[2:end-1], inner=2), seq[end])

In [ ]:
F = sparse(row_i, col_i, Fval, N, S) / 2 / Δx

In [54]:
N = 4
S = N*(N+1)/2
dx = 1/6

ii = repeat(2:(N-1), inner=4)
m = 2:N-1
mi = Int.([N*(N+1)/2 - (N-m)*(N-m+1)/2 - (N-m) for m in 2:N-1])               # this is where the xi^2 term is
mm = Int.([N*(N+1)/2 - (N-m).*(N-m+1)/2 - (N-m) - (N-(m-2)) for m in 2:N-1])  # this is where the x_{i-1}^2 term is
mp = Int.([N*(N+1)/2 - (N-m).*(N-m+1)/2 - (N-m) + (N-(m-1)) for m in 2:N-1])  # this is where the x_{i+1}^2 term is
jp = mi .+ 1  # this is the index of the x_{i+1}*x_i term
jm = mm .+ 1  # this is the index of the x_{i-1}*x_i term
jj = reshape([mp'; mm'; jp'; jm'],4*N-8);
vv = reshape([-ones(1,N-2); ones(1,N-2); -ones(1,N-2); ones(1,N-2)],4*N-8)/(6*dx);
F = sparse(ii,jj,vv,N,N*(N+1)/2)
# Boundary conditions
F[1,2] = -1/6/dx
F[1,N+1] = -1/6/dx
F[N,end-1] = 1/6/dx
F[N,end-2] = 1/6/dx
F

4×10 SparseMatrixCSC{Float64, Int64} with 12 stored entries:
  ⋅   -1.0   ⋅    ⋅   -1.0    ⋅    ⋅     ⋅     ⋅     ⋅ 
 1.0   1.0   ⋅    ⋅     ⋅   -1.0   ⋅   -1.0    ⋅     ⋅ 
  ⋅     ⋅    ⋅    ⋅    1.0   1.0   ⋅     ⋅   -1.0  -1.0
  ⋅     ⋅    ⋅    ⋅     ⋅     ⋅    ⋅    1.0   1.0    ⋅ 

In [64]:
a = range(0.8,1.2,10)

0.8:0.044444444444444446:1.2

In [67]:
zeros(1,1) .* rand(5,5)

5×5 Matrix{Float64}:
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0

In [78]:
a  = zeros(1,1)
a[1] == 0
a = size(a)==(1,1) && a[1]==0 ? 0 : 1

0

In [86]:
typeof(NaN) == Float64

true

In [3]:
a = 1 
b = 2
c = 3 
n = 10
@info """

Constraint         = Non-Constrained
order              = $(n)
termination_status = $(a)
primal_status      = $(b)
objective_value    = $(c)
"""

┌ Info: 
│ Constraint        = Non-Constrained
│ order              = 10
│ termination_status = 1
│ primal_status      = 2
│ objective_value    = 3
└ @ Main c:\Users\tkoike3\Code\LiftAndLearn.jl\examples\scribble.ipynb:5


In [21]:
function generateEPmatrix(N, Δx, μ::Float64)
    # Create A matrix
    A = diagm(0 => (-2) * ones(N), 1 => ones(N - 1), -1 => ones(N - 1)) * μ / Δx^2
    A[1, N] = μ / Δx^2
    A[N, 1] = μ / Δx^2

    # Create F matrix
    S = Int(N * (N + 1) / 2)
    if N >= 3
        ii = repeat(2:(N-1), inner=4)
        m = 2:N-1
        mi = Int.([N*(N+1)/2 - (N-m)*(N-m+1)/2 - (N-m) for m in 2:N-1])               # this is where the xi^2 term is
        mm = Int.([N*(N+1)/2 - (N-m).*(N-m+1)/2 - (N-m) - (N-(m-2)) for m in 2:N-1])  # this is where the x_{i-1}^2 term is
        mp = Int.([N*(N+1)/2 - (N-m).*(N-m+1)/2 - (N-m) + (N-(m-1)) for m in 2:N-1])  # this is where the x_{i+1}^2 term is
        jp = mi .+ 1  # this is the index of the x_{i+1}*x_i term
        jm = mm .+ 1  # this is the index of the x_{i-1}*x_i term
        jj = reshape([mp'; mm'; jp'; jm'],4*N-8);
        vv = reshape([-ones(1,N-2); ones(1,N-2); -ones(1,N-2); ones(1,N-2)],4*N-8)/(6*Δx);
        F = sparse(ii,jj,vv,N,S)

        # Boundary conditions
        F[1,2] = -1/6/Δx
        F[1,N+1] = -1/6/Δx
        F[1,N] = 1/6/Δx
        F[1,end] = 1/6/Δx
        F[N,end-1] = 1/6/Δx
        F[N,end-2] = 1/6/Δx
        F[N,1] = -1/6/Δx
        F[N,N] = -1/6/Δx
    else
        F = zeros(N, S)
    end

    return A, F
end

generateEPmatrix (generic function with 1 method)

In [24]:
A, F = generateEPmatrix(4, 1/6, 1.0)

([-72.0 36.0 0.0 36.0; 36.0 -72.0 36.0 0.0; 0.0 36.0 -72.0 36.0; 36.0 0.0 36.0 -72.0], sparse([2, 4, 1, 2, 1, 4, 1, 3, 2, 3, 2, 4, 3, 4, 1, 3], [1, 1, 2, 2, 4, 4, 5, 5, 6, 6, 8, 8, 9, 9, 10, 10], [1.0, -1.0, -1.0, 1.0, 1.0, -1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, -1.0, 1.0, 1.0, -1.0], 4, 10))

In [25]:
F

4×10 SparseMatrixCSC{Float64, Int64} with 16 stored entries:
   ⋅   -1.0   ⋅    1.0  -1.0    ⋅    ⋅     ⋅     ⋅    1.0
  1.0   1.0   ⋅     ⋅     ⋅   -1.0   ⋅   -1.0    ⋅     ⋅ 
   ⋅     ⋅    ⋅     ⋅    1.0   1.0   ⋅     ⋅   -1.0  -1.0
 -1.0    ⋅    ⋅   -1.0    ⋅     ⋅    ⋅    1.0   1.0    ⋅ 

In [20]:
A

5×5 Matrix{Float64}:
 -72.0   36.0    0.0    0.0   36.0
  36.0  -72.0   36.0    0.0    0.0
   0.0   36.0  -72.0   36.0    0.0
   0.0    0.0   36.0  -72.0   36.0
  36.0    0.0    0.0   36.0  -72.0